In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
import random
from sklearn import metrics
import time


In [2]:
class Reservoir(object):
    """
    Build a reservoir and evaluate internal states
    
    Parameters:
        n_internal_units = processing units in the reservoir
        spectral_radius = largest eigenvalue of the reservoir matrix of connection weights
        leak = amount of leakage in the reservoir state update (optional)
        connectivity = percentage of nonzero connection weights (unused in circle reservoir)
        input_scaling = scaling of the input connection weights
        noise_level = deviation of the Gaussian noise injected in the state update
        circle = generate determinisitc reservoir with circle topology
    """
    
    def __init__(self, n_internal_units=100, spectral_radius=0.99, leak=None,
                 connectivity=0.3, input_scaling=0.2, noise_level=0.01, circle=False):
        
        # Initialize attributes
        self._n_internal_units = n_internal_units
        self._input_scaling = input_scaling
        self._noise_level = noise_level
        self._leak = leak

        # Input weights depend on input size: they are set when data is provided
        self._input_weights = None

        # Generate internal weights
        if circle:
            self._internal_weights = self._initialize_internal_weights_Circ(
                    n_internal_units,
                    spectral_radius)
        else:
            self._internal_weights = self._initialize_internal_weights(
                n_internal_units,
                connectivity,
                spectral_radius)


    def _initialize_internal_weights_Circ(self, n_internal_units, spectral_radius):
        
        internal_weights = np.zeros((n_internal_units, n_internal_units))
        internal_weights[0,-1] = spectral_radius
        for i in range(n_internal_units-1):
            internal_weights[i+1,i] = spectral_radius
                
        return internal_weights
    
    
    def _initialize_internal_weights(self, n_internal_units,
                                     connectivity, spectral_radius):

        # Generate sparse, uniformly distributed weights.
        internal_weights = sparse.rand(n_internal_units,
                                       n_internal_units,
                                       density=connectivity).todense()

        # Ensure that the nonzero values are uniformly distributed in [-0.5, 0.5]
        internal_weights[np.where(internal_weights > 0)] -= 0.5
        
        # Adjust the spectral radius.
        E, _ = np.linalg.eig(internal_weights)
        e_max = np.max(np.abs(E))
        internal_weights /= np.abs(e_max)/spectral_radius       

        return internal_weights


    def _compute_state_matrix(self, X, n_drop=0):
        N, T, _ = X.shape
        previous_state = np.zeros((N, self._n_internal_units), dtype=float)

        # Storage
        state_matrix = np.empty((N, T - n_drop, self._n_internal_units), dtype=float)
        for t in range(T):
            current_input = X[:, t, :]

            # Calculate state
            state_before_tanh = self._internal_weights.dot(previous_state.T) + self._input_weights.dot(current_input.T)

            # Add noise
            state_before_tanh += np.random.rand(self._n_internal_units, N)*self._noise_level

            # Apply nonlinearity and leakage (optional)
            if self._leak is None:
                previous_state = np.tanh(state_before_tanh).T
            else:
                previous_state = (1.0 - self._leak)*previous_state + np.tanh(state_before_tanh).T

            # Store everything after the dropout period
            if (t > n_drop - 1):
                state_matrix[:, t - n_drop, :] = previous_state

        return state_matrix


    def get_states(self, X, n_drop=0, bidir=True):
        N, T, V = X.shape
        if self._input_weights is None:
            self._input_weights = (2.0*np.random.binomial(1, 0.5 , [self._n_internal_units, V]) - 1.0)*self._input_scaling

        # compute sequence of reservoir states
        states = self._compute_state_matrix(X, n_drop)
    
        # reservoir states on time reversed input
        if bidir is True:
            X_r = X[:, ::-1, :]
            states_r = self._compute_state_matrix(X_r, n_drop)
            states = np.concatenate((states, states_r), axis=2)

        return states
    
    def getReservoirEmbedding(self, X,pca, ridge_embedding,  n_drop=5, bidir=True, test = False):

        res_states = self.get_states(X, n_drop=5, bidir=True)


        N_samples = res_states.shape[0]
        res_states = res_states.reshape(-1, res_states.shape[2])                   
        # ..transform..
        if test:
            red_states = pca.transform(res_states)
        else:
            red_states = pca.fit_transform(res_states)          
        # ..and put back in tensor form
        red_states = red_states.reshape(N_samples,-1,red_states.shape[1])  

        coeff_tr = []
        biases_tr = []   

        for i in range(X.shape[0]):
            ridge_embedding.fit(red_states[i, 0:-1, :], red_states[i, 1:, :])
            coeff_tr.append(ridge_embedding.coef_.ravel())
            biases_tr.append(ridge_embedding.intercept_.ravel())
        print(np.array(coeff_tr).shape,np.array(biases_tr).shape)
        input_repr = np.concatenate((np.vstack(coeff_tr), np.vstack(biases_tr)), axis=1)
        return input_repr


In [3]:
tab = []
tt = []
dtyp = 'ddgz_60'

# ============ Load dataset ============
x = np.load('../data/training_data/' + dtyp + '_input_x.npy')
y = np.load('../data/training_data/' + dtyp + '_input_y.npy')
result = np.where(y==1) # Identify take over instances
all_indices = list(range(y.shape[0]))
ntc_index = [i for i in all_indices if i not in result[0]] # Identify non-take over instances
tc_index = list(result[0])
# Take a subset of the non-take over instances such that class ratio is 1:1
tc_idx_size = len(tc_index)
random.shuffle(ntc_index)
ntc_data_idx = ntc_index[:tc_idx_size]
# Combine and shuffle the indices
idx = ntc_data_idx+tc_index
random.shuffle(idx)
X_p = x[idx]
Y_p = y[idx]
Y_p = Y_p.reshape(-1,1)
pca = PCA(n_components=60)  
ridge_embedding = Ridge(alpha=10, fit_intercept=True)
readout = Ridge(alpha=5)

res = Reservoir(n_internal_units=400, spectral_radius=0.59, leak=None,
                 connectivity=0.2, input_scaling=0.1, noise_level=0.01, circle=False)

onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

for i in range(10):
    X, Xte, Y, Yte = train_test_split(X_p, Y_p, test_size=0.25, shuffle=True)
    # One-hot encoding for labels

    Y = onehot_encoder.fit_transform(Y)
    Yte = onehot_encoder.transform(Yte)

    start = time.time()
    input_repr = res.getReservoirEmbedding(X,pca, ridge_embedding,  n_drop=5, bidir=True, test = False)
    input_repr_te = res.getReservoirEmbedding(Xte,pca, ridge_embedding,  n_drop=5, bidir=True, test = True)

    readout.fit(input_repr, Y)

    end = time.time()
    logits = readout.predict(input_repr_te)
    pred_class = np.argmax(logits, axis=1)
    true_class = np.argmax(Yte, axis=1)
    #calculating the metrics
    cf = metrics.confusion_matrix(true_class, pred_class)
    total=sum(sum(cf))
    accuracy = (cf[0,0]+cf[1,1])/total
    specificity = cf[0,0]/(cf[0,0]+cf[0,1])
    sensitivity = cf[1,1]/(cf[1,0]+cf[1,1]) #recall
    precision = cf[1,1]/(cf[0,1]+cf[1,1])
    f1 = 2*(1/((1/precision)+(1/sensitivity)))
    ti = end - start

    tp = [accuracy, specificity, sensitivity, precision, f1, ti]
    tab.append(tp)

metric = pd.DataFrame(tab, columns=['accuracy', 'specificity', 'sensitivity', 'precision', 'f1-score', 'train_time'])

f_accuracy = metric['accuracy'].mean()
f_specificity = metric['specificity'].mean()
f_sensitivity = metric['sensitivity'].mean() #recall
f_precision = metric['precision'].mean()
f_f1 = metric['f1-score'].mean()
f_time = metric['train_time'].mean()
tle = [dtyp, f_accuracy, f_specificity, f_sensitivity, f_precision, f_f1, f_time]
tt.append(tle)

table = pd.DataFrame(tt, columns=['datatype', 'accuracy', 'specificity', 'sensitivity', 'precision', 'f1-score', 'train_time'])
table.to_excel('../data/results/accuracy_tables/'+ dtyp +'_echo_state.xlsx', index=False)


(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
(12436, 3600) (12436, 60)
(4146, 3600) (4146, 60)
